In [1]:
print("ok")

ok


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
import pinecone
import os
from langchain.document_loaders import PyMuPDFLoader,PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm.autonotebook import tqdm

c:\Users\Hp\Desktop\Medical-chatbot-llama2\mchatbot\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key=os.getenv("OPENAI_API_KEY")
pinecone_key=os.getenv("PINECONE_API_KEY")

In [4]:
os.environ["OPENAI_API_KEY"]=api_key
os.environ["PINECONE_API_KEY"]=pinecone_key


In [5]:
!pip install pinecone-client[grpc]

  Using cached grpc_gateway_protoc_gen_openapiv2-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached grpc_gateway_protoc_gen_openapiv2-0.1.0-py3-none-any.whl (12 kB)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3


DEPRECATION: Loading egg at c:\users\hp\desktop\medical-chatbot-llama2\mchatbot\lib\site-packages\medical_chatbot-0.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
protoc-gen-openapiv2 0.0.1 requires protobuf>=4.21.0, but you have protobuf 3.20.3 which is incompatible.


In [72]:
## Connect to Pinecone server
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone()

index_name = "medical-chatbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 

In [56]:
def load_pdf(directory):
    loader=PyPDFDirectoryLoader(directory,glob='*.pdf')
    data=loader.load()
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
    documents=text_splitter.split_documents(data)
    return documents

In [58]:
!pip install pypdf

  Using cached pypdf-4.2.0-py3-none-any.whl.metadata (7.4 kB)
Using cached pypdf-4.2.0-py3-none-any.whl (290 kB)



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [59]:
extracted_data=load_pdf(r'C:\Users\Hp\Desktop\Medical-chatbot-llama2\data')
extracted_data[0].page_content

'TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION'

In [65]:
print(extracted_data[150].page_content)

example, she is at higher risk for complications during preg-nancy because of a disorder or disease such as diabetes.
However, selective reduction is recommended often in
cases of multi-fetal pregnancy, or the presence of more thanone fetus, typically, at least three or more fetuses. In thegeneral population, multi-fetal pregnancy happens in onlyabout 1-2% of pregnant women. But multi-fetal pregnan-cies occur far more often in women using fertility drugs.
Precautions


In [67]:
embeddings=OpenAIEmbeddings()

In [68]:

!pip install langchain_pinecone

  Using cached langchain_pinecone-0.1.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached pinecone_client-3.2.2-py3-none-any.whl.metadata (16 kB)
Using cached langchain_pinecone-0.1.1-py3-none-any.whl (8.4 kB)
Using cached pinecone_client-3.2.2-py3-none-any.whl (215 kB)
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 4.1.1
    Uninstalling pinecone-client-4.1.1:
      Successfully uninstalled pinecone-client-4.1.1



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [73]:
from langchain_pinecone import PineconeVectorStore
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [75]:
vectorstore_from_docs = PineconeVectorStore.from_documents(
        extracted_data,
        index_name=index_name,
        embedding=embeddings
    )

In [76]:
# Ranked Results
query="What are allergies"
docs=vectorstore_from_docs.similarity_search(query,k=3)
print(docs)

[Document(page_content='Allergy and Immunology. Boston: Little, Brown and Co.,\n1995.\nNovick, N. L. You Can Do Something About Your Allergies.\nNew York: Macmillan, 1994.\nWeil, A. Natural Health, Natural Medicine: A Comprehensive\nManual for Wellness and Self-Care. New York: Houghton\nMifflin, 1995.\nRichard Robinson\nAllergies\nDefinition\nAllergies are abnormal reactions of the immune sys-\ntem that occur in response to otherwise harmless sub-stances.', metadata={'page': 127.0, 'source': 'C:\\Users\\Hp\\Desktop\\Medical-chatbot-llama2\\data\\Medical_book.pdf'}), Document(page_content='Description\nAllergies are among the most common of medical\ndisorders. It is estimated that 60 million Americans, ormore than one in every five people, suffer from someform of allergy, with similar proportions throughoutmuch of the rest of the world. Allergy is the single largestreason for school absence and is a major source of lostproductivity in the workplace.\nAn allergy is a type of immune react

In [77]:
# Prompt Template
prompt_template='''
Use the following pieces of information to answer the users questions.
if you dont know the answer , just say that you dont know . dont try to 
make up the answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful Answer :
'''

In [78]:
PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [79]:
llm=OpenAI()

In [1]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vectorstore_from_docs.as_retriever(),
    return_source_documents=True
)

NameError: name 'RetrievalQA' is not defined

In [1]:
while True:
    user_input=input(f'Input prompt')
    results=qa({'query':user_input })
    print("Response: ",results['result'])

NameError: name 'qa' is not defined

In [6]:
!pip install pinecone

DEPRECATION: Loading egg at c:\users\hp\desktop\medical-chatbot-llama2\mchatbot\lib\site-packages\medical_chatbot-0.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


   ---------------------------------------- 0.0/214.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/214.4 kB ? eta -:--:--
   ----- --------------------------------- 30.7/214.4 kB 660.6 kB/s eta 0:00:01
   ----- --------------------------------- 30.7/214.4 kB 660.6 kB/s eta 0:00:01
   ------------------ ------------------- 102.4/214.4 kB 590.8 kB/s eta 0:00:01
   ------------------ ------------------- 102.4/214.4 kB 590.8 kB/s eta 0:00:01
   --------------------- ---------------- 122.9/214.4 kB 481.4 kB/s eta 0:00:01
   --------------------------- ---------- 153.6/214.4 kB 510.2 kB/s eta 0:00:01
   ----------------------------- -------- 163.8/214.4 kB 468.3 kB/s eta 0:00:01
   -------------------------------------- 214.4/214.4 kB 523.5 kB/s eta 0:00:00


In [8]:
from pinecone import Pinecone

In [ ]:
Pinecone.fr